In [181]:
# Packages for Web-Scraping
from requests import get
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn

# Packages for Saving File after Scraping
import numpy as np
import pandas as pd

In [156]:
url = 'https://www.politifact.com/truth-o-meter/statements/?page=1'
response = get(url)
print(response.text[:500])


<!DOCTYPE html>
<html xmlns="https://www.w3.org/1999/xhtml" xml:lang="en" lang="en" xmlns:og="https://ogp.me/ns#" xmlns:fb="https://www.facebook.com/2008/fbml">
<head>


<title>Our latest fact-checks | PolitiFact</title>
<meta http-equiv="Content-type" content="text/html;charset=UTF-8" />
<meta name="description" content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." />
<meta name="twitter:username" content="Politi


In [157]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [158]:
statement_containers = html_soup.find_all('div', class_ = 'scoretable__item')
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
20


In [159]:
s = statement_containers[1]
s

<div class="scoretable__item">
<div class="statement">
<div class="meter">
<a href="/personalities/viral-image/statements/byruling/false/">
<img alt="False" src="https://static.politifact.com/CACHE/images/rulings/tom-false/e5c0d30b7228205f896c99d3216f498f.jpg"/>
</a>
<p class="quote">Some advice? This quote is apocryphal</p>
</div>
<div class="statement__body">
<div class="mugshot">
<a href="/personalities/viral-image/">
<img alt=" Viral image" src="https://static.politifact.com/CACHE/images/politifact/mugs/viral-pictures/e3608ba2adb7d0457800681cb7df4918.jpg"/>
</a>
</div>
<div class="statement__source">
<a href="/personalities/viral-image/">
Viral image
</a>
</div>
<p class="statement__text">
<a class="link" href="/facebook-fact-checks/statements/2019/oct/25/viral-image/no-evidence-ann-landers-said-quote-about-age/">Says Ann Landers said, "At age 20, we worry about what others think of us. At age 40, we don’t care what they think. At age 60, we discover that they have not been thinkin

In [160]:
s.find('p', class_ = 'statement__text').get_text(strip=True)

'Says Ann Landers said, "At age 20, we worry about what others think of us. At age 40, we don’t care what they think. At age 60, we discover that they have not been thinking of us at all."'

In [161]:
s.find('div', class_ = 'statement__source').get_text(strip=True)

'Viral image'

In [162]:
s.find('p', class_ = 'statement__text').a["href"]

'/facebook-fact-checks/statements/2019/oct/25/viral-image/no-evidence-ann-landers-said-quote-about-age/'

In [163]:
s.img["alt"]

'False'

In [165]:
# Lists to store the scraped data in
statement = []
source = []
link = []
veracity = []

# Extract data from individual container
for container in statement_containers:
# statement
    sta = container.find('p', class_ = 'statement__text').get_text(strip=True)
    statement.append(sta)
# source
    sou = container.find('div', class_ = 'statement__source').get_text(strip=True)
    source.append(sou)
# link
    lin = container.find('p', class_ = 'statement__text').a["href"]
    link.append(lin)
# veracity
    ver = container.img["alt"]
    veracity.append(ver)

In [166]:
test_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
statement    20 non-null object
source       20 non-null object
link         20 non-null object
veracity     20 non-null object
dtypes: object(4)
memory usage: 720.0+ bytes
None


,statement,source,link,veracity
0,"Says that in 1770 ""British parliament banned lipstick, saying it had the power to seduce men into marriage, which was classified as witchcraft.""",Facebook posts,/facebook-fact-checks/statements/2019/oct/25/facebook-posts/no-british-parliament-didnt-ban-witchcraft-lipstic/,Pants on Fire!
1,"Says Ann Landers said, ""At age 20, we worry about what others think of us. At age 40, we don’t care what they think. At age 60, we discover that they have not been thinking of us at all.""",Viral image,/facebook-fact-checks/statements/2019/oct/25/viral-image/no-evidence-ann-landers-said-quote-about-age/,False
2,"""General Motors is making record profits.""",Glenn Kage,/missouri/statements/2019/oct/25/glenn-kage/general-motors-profits-have-been-record-setting-no/,Half-True
3,"""14,000 abandoned wind turbines litter the United States.""",Chain email,/missouri/statements/2019/oct/25/chain-email/no-there-arent-14000-abandoned-wind-turbines-litte/,Pants on Fire!
4,"Says Congress gave Wall Street ""trillions of dollars of zero interest loans.""",Bernie Sanders,/truth-o-meter/statements/2019/oct/25/bernie-sanders/bernie-sanders-overstates-recession-aid-wall-stree/,Half-True
5,"Says Chick-fil-A funds the National Christian Foundation, which ""paid a preacher to go to Uganda and help their lawmakers"" with a bill that would impose the death penalty for homosexuality.",Facebook posts,/facebook-fact-checks/statements/2019/oct/25/facebook-posts/what-does-chick-fil-have-do-ugandas-anti-gay-bill-/,Mostly False
6,"Says Donald Trump said, ""Make no mistake: They’re not after me. They are after you; your guns, your Christian values, your freedom. I’m just in their way.""",Viral image,/facebook-fact-checks/statements/2019/oct/25/viral-image/no-donald-trump-didnt-say-about-christian-values-a/,False
7,"""President Trump is asking everyone to forward this email… The TRUMP Rules: Congressional Reform Act of 2017.""",Viral image,/facebook-fact-checks/statements/2019/oct/25/viral-image/no-trump-isnt-asking-you-forward-email-about-trump/,Pants on Fire!
8,"Says his impeachment is a ""coup.""",Donald Trump,/truth-o-meter/statements/2019/oct/25/donald-trump/donald-trump-says-impeachment-coup-s-pants-fire/,Pants on Fire!
9,"""Barack Obama put a permanent stop on this military aid to the Ukraine - he never allowed it to go.""",Matt Gaetz,/florida/statements/2019/oct/25/matt-gaetz/matt-gaetz-says-obama-permanently-stopped-military/,Mostly False


In [178]:
pages = [str(i) for i in range(1,5)]
start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.4986879180622635 requests/s
Request: 2; Frequency: 0.3991497983221507 requests/s
Request: 3; Frequency: 0.4988702292400658 requests/s
Request: 4; Frequency: 0.443620000481241 requests/s
Request: 5; Frequency: 0.41598240504109535 requests/s


In [180]:
start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.4156157703991761 requests/s


In [185]:
# Lists to store the scraped data in
statement = []
source = []
link = []
veracity = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every page in the interval 1-4
pages = [str(i) for i in range(1,834)]
for page in pages:

    # Make a get request
    response = get('https://www.politifact.com/truth-o-meter/statements/?page=' + page)

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
#     if requests > 100:
#         warn('Number of requests was greater than expected.')
#         break

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the 50 movie containers from a single page
    statement_containers = page_html.find_all('div', class_ = 'scoretable__item')

    for container in statement_containers:
    # statement
        sta = container.find('p', class_ = 'statement__text').get_text(strip=True)
        statement.append(sta)
    # source
        sou = container.find('div', class_ = 'statement__source').get_text(strip=True)
        source.append(sou)
    # link
        lin = container.find('p', class_ = 'statement__text').a["href"]
        link.append(lin)
    # veracity
        ver = container.img["alt"]
        veracity.append(ver)

Request:833; Frequency: 0.07729913196146226 requests/s


In [186]:
politifact_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(politifact_df.info())
politifact_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16611 entries, 0 to 16610
Data columns (total 4 columns):
statement    16611 non-null object
source       16611 non-null object
link         16611 non-null object
veracity     16611 non-null object
dtypes: object(4)
memory usage: 519.2+ KB
None


,statement,source,link,veracity
16609,"""Tommy Thompson is the father of welfare reform, creating the groundbreaking Wisconsin Works program in Wisconsin that became the model for national and international reforms.""",Tommy Thompson,/truth-o-meter/statements/2007/jun/01/tommy-thompson/thompson-was-a-leader-in-welfare-reform/,True
16610,"""Tommy Thompson created the first school choice program in the nation, giving thousands of Milwaukee students the choice of where they go to school – no matter where they live or how much money th...",Tommy Thompson,/truth-o-meter/statements/2007/may/02/tommy-thompson/he-was-credited-as-a-tinkerer/,True


In [187]:
politifact_df.to_csv('politifact.csv')

In [188]:
politifact_df.source.value_counts()

Donald Trump                                                   734
Barack Obama                                                   620
Bloggers                                                       484
Hillary Clinton                                                302
Facebook posts                                                 268
Viral image                                                    239
Mitt Romney                                                    216
Scott Walker                                                   214
John McCain                                                    190
Chain email                                                    181
Rick Perry                                                     176
Rick Scott                                                     169
Marco Rubio                                                    163
Bernie Sanders                                                 151
Ted Cruz                                                      

In [193]:
politifact_df[politifact_df.source=="Donald Trump"]

,statement,source,link,veracity
8,"Says his impeachment is a ""coup.""",Donald Trump,/truth-o-meter/statements/2019/oct/25/donald-trump/donald-trump-says-impeachment-coup-s-pants-fire/,Pants on Fire!
35,"""The Kurds are much safer right now.""",Donald Trump,/truth-o-meter/statements/2019/oct/16/donald-trump/after-attacks-turkey-trump-falsely-claims-kurds-sy/,False
58,"""Fact: Joe Biden pressured Ukraine to fire its prosecutor .... Fact: the prosecutor said he was forced out for leading a corruption probe into Hunter Biden's company. Fact: Democrats want to impea...",Donald Trump,/truth-o-meter/statements/2019/oct/11/donald-trump/trump-ad-misleads-about-biden-ukraine-and-prosecut/,False
63,"Says Republican Dan Bishop was ""down by 17"" points in NC congressional race and then won ""by a lot.""",Donald Trump,/north-carolina/statements/2019/oct/10/donald-trump/trump-wrong-about-north-carolina-congressional-ele/,False
66,"""The first so-called second hand information ‘Whistleblower’ got my phone conversation almost completely wrong.""",Donald Trump,/truth-o-meter/statements/2019/oct/09/donald-trump/donald-trump-whistleblower-got-phone-call-almost-c/,Pants on Fire!
70,"Says Joe Biden ""said he never spoke to the Ukrainian company, and then the picture came out where he was playing golf with the company boss and Hunter"" Biden.",Donald Trump,/truth-o-meter/statements/2019/oct/08/donald-trump/trump-claims-photo-shows-bidens-playing-golf-ukrai/,False
71,"Minneapolis ""city government preemptively informed the Target Center that it would be responsible for $530,000 in security and other costs"" related to a Trump rally, ""more than 26 times the estima...",Donald Trump,/truth-o-meter/statements/2019/oct/08/donald-trump/trump-campaign-says-minneapolis-wants-charge-53000/,True
89,"Says in the 2016 presidential election, ""I won that one by 42 points -- 42 points -- West Virginia.""",Donald Trump,/west-virginia/statements/2019/oct/02/donald-trump/did-donald-trump-win-west-virginia-42-points-2016/,True
96,"""Longstanding whistleblower rules (were changed) just before submittal of the fake whistleblower report.""",Donald Trump,/truth-o-meter/statements/2019/oct/01/donald-trump/donald-trumps-false-claim-about-change-whistleblow/,False
102,"""For the first time in history, most new hires of prime working age are minorities and women.""",Donald Trump,/truth-o-meter/statements/2019/sep/27/donald-trump/fact-checking-trump-hiring-minorities-women/,Half-True


In [194]:
politifact_df.veracity.value_counts()

False             3274
Half-True         3158
Mostly True       3010
Mostly False      2756
True              2340
Pants on Fire!    1817
Full Flop          159
Half Flip           70
No Flip             27
Name: veracity, dtype: int64